# Unknow feature

### I would like to figure out what is "geopotential_height_cloud_base"

In [1]:
%%time

from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType


feats = []
f = open('../features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

df = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:13030/nam_s/*')
df.show(3)

+-------------+------------+------------------------+---------------------------------------+------------------------+------------------------------+----------------------------------------+------------------+------------------------------------------+------------------------------------------+---------------------------+----------------------------------+-----------------------------------+---------------------------------+------------------------------+-------------------------+-----------------------------------+---------------------------------+-----------------------------+---------------------------------+-------------------------+--------------+-----------------+------------------------------+-----------------------------+----------------+------------------------------------------------+---------------------+------------------+---------------------------------+-----------------------------------+---------------------------------------+------------------------------+-------------

In [4]:
df.describe(['geopotential_height_cloud_base']).show()

+-------+------------------------------+
|summary|geopotential_height_cloud_base|
+-------+------------------------------+
|  count|                     108000000|
|   mean|            -364.1897130100188|
| stddev|             3283.338557541059|
|    min|                       -5000.0|
|    max|                       14828.5|
+-------+------------------------------+



* I assume it's the height of the cloud, but there are negative numbers which is really weird.
* [Geopotential height](https://en.wikipedia.org/wiki/Geopotential_height) is a vertical coordinate referenced to Earth's mean sea level. One usually speaks of the geopotential height of a certain pressure level, which would correspond to the geopotential height at which that pressure occurs.
* [Cloud base](https://en.wikipedia.org/wiki/Cloud_base) is the lowest altitude of the visible portion of the cloud.
* Combined two terms above, I got the idea that geopotential height cloud base is the lowest pressure level of the visible portion of the cloud. So the negative one could be in the air. The ones with high values could be in valley and I am not sure if fog counts as cloud. Overall, the mean is negative that indicates most of the time the cloud base is in the sky (very high since pressure is low).